In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
import warnings
warnings.filterwarnings('ignore')
from matplotlib import font_manager,rc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from fbprophet import Prophet 

In [37]:
data=pd.read_csv("/content/drive/MyDrive/찌끄래기/final_dataset.csv")

In [38]:
data.drop(['Unnamed: 0'],axis=1,inplace=True)

In [39]:
data

,year,month,date,emd_nm,em_g,total_pop,제주_resd_ratio,제주_resd_pop,visit_ratio,card_use_cnt,delivery_per_cnt,market_per_cnt,frgn_ratio,youth_ratio
0,2018,1,1,건입동,1708250,10028,0.552623,7274.890502,49.642958,1806.0,0.002769,0.326135,0.038427,0.200261
1,2018,1,1,남원읍,1239600,19740,0.704247,12143.821692,41.452047,1581.0,0.015813,0.294750,0.025900,0.177984
2,2018,1,1,노형동,9357900,54958,0.769356,50660.687565,25.674217,12507.0,0.009834,0.392180,0.039271,0.195624
3,2018,1,1,대륜동,1717700,13651,0.667224,11185.575639,42.502310,2032.0,0.009843,0.422736,0.014440,0.208504
4,2018,1,1,대정읍,1264950,22603,0.784751,16075.181182,28.652628,1973.0,0.017233,0.437912,0.053508,0.168114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50881,2021,6,30,한경면,973850,9495,0.749118,7257.895822,35.350145,1286.0,0.031104,0.328927,0.042559,0.171578
50882,2021,6,30,한림읍,3298100,24462,0.769060,20632.727912,35.614095,3044.0,0.084428,0.219777,0.048140,0.197206
50883,2021,6,30,화북동,3749600,24630,0.705806,24117.268821,27.871984,5644.0,0.114103,0.291460,0.010642,0.167519
50884,2021,6,30,효돈동,1011050,5314,0.744489,3726.154500,31.896263,714.0,0.021008,0.450980,0.014730,0.165783


In [40]:
data['year']=data['year'].astype(str)
data['month']=data['month'].astype(str)
data['date']=data['date'].astype(str)

In [41]:
data['base_date']=data['year']+"-"+data['month']+"-"+data['date']

In [42]:
data['base_date']=pd.to_datetime(data['base_date'])

In [43]:
data.drop(['year','month','date'],axis=1,inplace=True)

In [44]:
data.head()

,emd_nm,em_g,total_pop,제주_resd_ratio,제주_resd_pop,visit_ratio,card_use_cnt,delivery_per_cnt,market_per_cnt,frgn_ratio,youth_ratio,base_date
0,건입동,1708250,10028,0.552623,7274.890502,49.642958,1806.0,0.002769,0.326135,0.038427,0.200261,2018-01-01
1,남원읍,1239600,19740,0.704247,12143.821692,41.452047,1581.0,0.015813,0.294750,0.025900,0.177984,2018-01-01
2,노형동,9357900,54958,0.769356,50660.687565,25.674217,12507.0,0.009834,0.392180,0.039271,0.195624,2018-01-01
3,대륜동,1717700,13651,0.667224,11185.575639,42.502310,2032.0,0.009843,0.422736,0.014440,0.208504,2018-01-01
4,대정읍,1264950,22603,0.784751,16075.181182,28.652628,1973.0,0.017233,0.437912,0.053508,0.168114,2018-01-01


# card_use_cnt 예측

In [45]:
test_df=pd.DataFrame(columns=['emd_nm','base_date','card_use_cnt'])

In [46]:
for region in data['emd_nm'].unique():
    data_pop=data.loc[data['emd_nm']==region,['card_use_cnt','base_date']]
    data_pop.rename(columns={'card_use_cnt':'y','base_date':'ds'},inplace=True)
    model=Prophet()
    model.daily_seasonality=True
    model.weekly_seasonality=True
    model.fit(data_pop)
    future=model.make_future_dataframe(periods=3,freq='M')
    forecast=model.predict(future)
    forecast=forecast[['ds','yhat']].tail(3)
    forecast.rename(columns={'ds':'base_date','yhat':'card_use_cnt'},inplace=True)
    forecast['emd_nm']=region
    test_df=pd.concat([test_df,forecast])

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


In [47]:
test_df

,emd_nm,base_date,card_use_cnt
1276,건입동,2021-07-31,2620.450508
1277,건입동,2021-08-31,2232.604802
1278,건입동,2021-09-30,2171.089782
1276,남원읍,2021-07-31,2362.084697
1277,남원읍,2021-08-31,2077.894464
...,...,...,...
600,한림읍,2021-08-31,2973.247067
601,한림읍,2021-09-30,2966.313077
597,한경면,2021-07-31,1083.340724
598,한경면,2021-08-31,1096.180816


# delivery_per_cnt 예측

In [48]:
test_ratio=pd.DataFrame(columns=['emd_nm','base_date','delivery_per_cnt'])

In [49]:
for region in data['emd_nm'].unique():
    data_ratio=data.loc[data['emd_nm']==region,['delivery_per_cnt','base_date']]
    data_ratio.rename(columns={'delivery_per_cnt':'y','base_date':'ds'},inplace=True)
    model=Prophet()
    model.fit(data_ratio)
    future=model.make_future_dataframe(periods=3,freq='M')
    forecast=model.predict(future)
    forecast=forecast[['ds','yhat']].tail(3)
    forecast.rename(columns={'ds':'base_date','yhat':'delivery_per_cnt'},inplace=True)
    forecast['emd_nm']=region
    test_ratio=pd.concat([test_ratio,forecast])

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

In [50]:
test_ratio[test_ratio['emd_nm'] == '건입동']

,emd_nm,base_date,delivery_per_cnt
1276,건입동,2021-07-31,0.128435
1277,건입동,2021-08-31,0.123999
1278,건입동,2021-09-30,0.127485


# market_per_cnt 예측

In [51]:
test_per=pd.DataFrame(columns=['emd_nm','base_date','market_per_cnt'])

In [52]:
for region in data['emd_nm'].unique():
    data_ratio=data.loc[data['emd_nm']==region,['market_per_cnt','base_date']]
    data_ratio.rename(columns={'market_per_cnt':'y','base_date':'ds'},inplace=True)
    model=Prophet()
    model.fit(data_ratio)
    future=model.make_future_dataframe(periods=3,freq='M')
    forecast=model.predict(future)
    forecast=forecast[['ds','yhat']].tail(3)
    forecast.rename(columns={'ds':'base_date','yhat':'market_per_cnt'},inplace=True)
    forecast['emd_nm']=region
    test_per=pd.concat([test_per,forecast])

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

In [53]:
test_per

,emd_nm,base_date,market_per_cnt
1276,건입동,2021-07-31,0.273041
1277,건입동,2021-08-31,0.296707
1278,건입동,2021-09-30,0.290987
1276,남원읍,2021-07-31,0.258271
1277,남원읍,2021-08-31,0.281904
...,...,...,...
600,한림읍,2021-08-31,0.225575
601,한림읍,2021-09-30,0.225885
597,한경면,2021-07-31,0.279432
598,한경면,2021-08-31,0.298511


# frgn_ratio 예측

In [54]:
test_frgn=pd.DataFrame(columns=['emd_nm','base_date','frgn_ratio'])

In [55]:
for region in data['emd_nm'].unique():
    data_ratio=data.loc[data['emd_nm']==region,['frgn_ratio','base_date']]
    data_ratio.rename(columns={'frgn_ratio':'y','base_date':'ds'},inplace=True)
    model=Prophet()
    model.fit(data_ratio)
    future=model.make_future_dataframe(periods=3,freq='M')
    forecast=model.predict(future)
    forecast=forecast[['ds','yhat']].tail(3)
    forecast.rename(columns={'ds':'base_date','yhat':'frgn_ratio'},inplace=True)
    forecast['emd_nm']=region
    test_frgn=pd.concat([test_frgn,forecast])

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

In [56]:
test_frgn

,emd_nm,base_date,frgn_ratio
1276,건입동,2021-07-31,0.026973
1277,건입동,2021-08-31,0.033025
1278,건입동,2021-09-30,0.033945
1276,남원읍,2021-07-31,0.031920
1277,남원읍,2021-08-31,0.031453
...,...,...,...
600,한림읍,2021-08-31,0.047989
601,한림읍,2021-09-30,0.047326
597,한경면,2021-07-31,0.043855
598,한경면,2021-08-31,0.044065


# youth ratio

In [57]:
test_youth=pd.DataFrame(columns=['emd_nm','base_date','youth_ratio'])

In [58]:
for region in data['emd_nm'].unique():
    data_ratio=data.loc[data['emd_nm']==region,['youth_ratio','base_date']]
    data_ratio.rename(columns={'youth_ratio':'y','base_date':'ds'},inplace=True)
    model=Prophet()
    model.fit(data_ratio)
    future=model.make_future_dataframe(periods=3,freq='M')
    forecast=model.predict(future)
    forecast=forecast[['ds','yhat']].tail(3)
    forecast.rename(columns={'ds':'base_date','yhat':'youth_ratio'},inplace=True)
    forecast['emd_nm']=region
    test_youth=pd.concat([test_youth,forecast])

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

In [59]:
test_youth

,emd_nm,base_date,youth_ratio
1276,건입동,2021-07-31,0.200937
1277,건입동,2021-08-31,0.202925
1278,건입동,2021-09-30,0.200979
1276,남원읍,2021-07-31,0.168607
1277,남원읍,2021-08-31,0.163844
...,...,...,...
600,한림읍,2021-08-31,0.207352
601,한림읍,2021-09-30,0.211116
597,한경면,2021-07-31,0.179695
598,한경면,2021-08-31,0.180984


In [60]:
merge=pd.merge(test_df,test_ratio)

In [61]:
merge=pd.merge(merge,test_per)

In [62]:
merge=pd.merge(merge,test_frgn)

In [63]:
merge=pd.merge(merge,test_youth)

In [64]:
merge.to_csv('/content/drive/MyDrive/찌끄래기/test_feature.csv')
merge

,emd_nm,base_date,card_use_cnt,delivery_per_cnt,market_per_cnt,frgn_ratio,youth_ratio
0,건입동,2021-07-31,2620.450508,0.128435,0.273041,0.026973,0.200937
1,건입동,2021-08-31,2232.604802,0.123999,0.296707,0.033025,0.202925
2,건입동,2021-09-30,2171.089782,0.127485,0.290987,0.033945,0.200979
3,남원읍,2021-07-31,2362.084697,0.062767,0.258271,0.031920,0.168607
4,남원읍,2021-08-31,2077.894464,0.065809,0.281904,0.031453,0.163844
...,...,...,...,...,...,...,...
121,한림읍,2021-08-31,2973.247067,0.090092,0.225575,0.047989,0.207352
122,한림읍,2021-09-30,2966.313077,0.096241,0.225885,0.047326,0.211116
123,한경면,2021-07-31,1083.340724,0.040944,0.279432,0.043855,0.179695
124,한경면,2021-08-31,1096.180816,0.034713,0.298511,0.044065,0.180984


In [65]:
total_region=merge.loc[merge['emd_nm']=='전체']

In [66]:
no=merge.loc[merge['emd_nm']!='전체'][['emd_nm', 'base_date', 'card_use_cnt', 'delivery_per_cnt', 'market_per_cnt', 'frgn_ratio', 'youth_ratio']].groupby(['base_date']).sum()

In [67]:
no.to_csv('/content/drive/MyDrive/찌끄래기/unknown_feature.csv')

In [34]:
no

,card_use_cnt,delivery_per_cnt,market_per_cnt,frgn_ratio,youth_ratio
base_date,,,,,
2018-01-01,99414.277752,0.249269,14.495064,1.260533,7.269386
2018-01-02,101017.096913,0.259908,14.060185,1.250017,7.199732
2018-01-03,101868.233125,0.248801,13.971141,1.241046,7.157023
2018-01-04,101263.144504,0.263591,13.997454,1.236420,7.163746
2018-01-05,104452.089468,0.365697,13.266841,1.235174,7.182139
...,...,...,...,...,...
2021-06-29,139039.219122,3.973634,12.497185,1.046147,7.961042
2021-06-30,140527.028666,4.058017,12.731529,1.072439,8.137652
2021-07-31,165961.984494,4.500282,11.927468,1.063815,8.189059
